<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- Aperture Photometry
    - Performing aperture photometry at multiple positions with the same aperture
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Performing aperture photometry at multiple positions using different apertures
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Bad pixel masking
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Encircled flux
      - Using the aperture_photometry function
      - Using the ApertureStats class

- ApertureMask objects
</div>

## Preliminaries

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

data = sci_hdulist[0].data.astype(float)
error = rms_hdulist[0].data.astype(float)
hdr = sci_hdulist[0].header
wcs = WCS(hdr)

In [ ]:
from astropy.visualization import simple_norm

plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
plt.title('XDF F160W Cutout');

# Aperture Photometry

## Performing aperture photometry at multiple positions with the same aperture

The aperture is the same size for each source.

*Note that the background has already been subtracted from this dataset.*

In [ ]:
import astropy.units as u
from photutils.utils import calc_total_error
from photutils.aperture import ApertureStats, CircularAperture, aperture_photometry

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.0
aperture = CircularAperture(positions, r=radius)

eff_gain = hdr['TEXPTIME']
total_error = calc_total_error(data, error, eff_gain)

unit = u.electron / u.s

### Using the `aperture_photometry` function

In [ ]:
phot = aperture_photometry(data << unit, aperture, error=total_error << unit)
phot

### Using the `ApertureStats` class

In [ ]:
apstats = ApertureStats(data << unit, aperture, error=total_error << unit)
apstats.sum, apstats.sum_err

In [ ]:
apstats.to_table()  # all source properties within the aperture

## Performing aperture photometry at multiple positions using multiple apertures

Photometry is measured for each of the three sources in four apertures of different radii.

We create separate `CircularAperture` object for each of the radii (with all three positions) and then create a list of these circular apertures.  The following example uses a Python list comprehension to create the list of apertures.

*Note that the background has already been subtracted from this dataset.*

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radii = [5.0, 7.5, 9.0, 11.0]
apertures = [CircularAperture(positions, r=r) for r in radii]
apertures

In [ ]:
apertures[2]  # the apertures for the third radius (r=9.0)

In [ ]:
apertures[2][1]  # the aperture for the third radius (r=9.0) at the second position

### Using `aperture_photometry`

In [ ]:
phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

The output table above now contains multiple columns for the `aperture_sum` and `aperture_sum_err` for each aperture.  The column names are appended with `_N`, where N is running index of the apertures in the input `apertures` list, i.e., the first aperture is `_0`, the second is `_1`, etc.

We could add columns to the table to store the value of the aperture radii used:

In [ ]:
for i, radius in enumerate(radii):
    phot[f'aperture_radius_{i}'] = np.ones(len(phot)) * radius * u.pix

phot

Since the column values are all the same, it's perhaps better to store the radii values in the table metadata:

In [ ]:
for i, radius in enumerate(radii):
    phot.meta[f'aperture_{i}'] = f'Circular aperture with r={radius} pix'

phot.meta

### Using `ApertureStats`

In [ ]:
apstats = []
for aper in apertures:
    apstat = ApertureStats(data << unit, aper, error=total_error << unit)
    apstat.meta['radius'] = apstat.aperture.r
    apstats.append(apstat)

for apstat in apstats:
    print(apstat.meta['radius'])

In [ ]:
for apstat in apstats:
    print(apstat.sum)

In [ ]:
for apstat in apstats:
    print(apstat.sum_err)

In [ ]:
apstats[0].to_table()  # all source properties within the first radius (r=5.0)

## Bad pixel masking

Pixels can be ignored/excluded (e.g., bad pixels) from the aperture photometry by providing a boolean (True/False) mask image via the mask keyword.

In [ ]:
# let's create a single bad pixel (with a large value) within the first source
data2 = data.copy()
y, x = 59, 91
data2[y, x] = 100.0

aperture_photometry(data2, aperture, error=total_error)

Note the large `aperture_sum` in the first source due to the bad pixel.

Now let's mask the bad pixel so that it does not contribute to the photometry.

In [ ]:
mask = np.zeros_like(data2, dtype=bool)
mask[y, x] = True  # True values are ignored

### Using `aperture_photometry`

In [ ]:
aperture_photometry(data2, aperture, error=total_error, mask=mask)

### Using `ApertureStats`

In [ ]:
apstats = ApertureStats(data2, aperture, error=total_error, mask=mask)
apstats.to_table()

## Encircled flux

For this example we will perform aperture photometry at a *single* position, but with *many* apertures of different radii.

Instead of generating a big table with many columns (one for each radius), we'll simply loop over the apertures and extract the fluxes from individual tables.

In this example, we manually compute the curve of growth using aperture photometry.  However, note that Photutils has a [Radial Profiles subpackage](https://photutils.readthedocs.io/en/latest/profiles.html) that makes it easy to compute a radial profile or a curve of growth.

### Using `aperture_photometry`

In [ ]:
radii = np.linspace(0.1, 20, 100)  # 100 radii from r=0.1 to 20 pixels
fluxes1 = []
for radius in radii:
    aper = CircularAperture(positions[1], r=radius)  # single position
    phot = aperture_photometry(data, aper)
    fluxes1.append(phot['aperture_sum'][0])

plt.plot(radii, fluxes1, 'o-')
plt.title('Encircled Flux')
plt.xlabel('Radius (pixels)')
plt.ylabel('Aperture Sum ($e^{-1}/s$)');

### Using `ApertureStats`

In [ ]:
radii = np.linspace(0.1, 20, 100)  # 100 radii from r=0.1 to 20 pixels
apstats = []
for radius in radii:
    aper = CircularAperture(positions[1], r=radius)  # single position
    apstats.append(ApertureStats(data, aper))

fluxes2 = []
areas = []
for apstat in apstats:
    fluxes2.append(apstat.sum)
    areas.append(apstat.sum_aper_area.value)

plt.plot(radii, fluxes2, 'o-')
plt.title('Encircled Flux')
plt.xlabel('Radius (pixels)')
plt.ylabel('Aperture Sum ($e^{-1}/s$)');

In [ ]:
plt.plot(radii, areas, 'o-')
plt.xlabel('Radius (pixels)')
plt.ylabel('Aperture Area ($pix^2$)');

# ApertureMask objects

All `PixelAperture` objects have a `to_mask` method that returns an [ApertureMask](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.ApertureMask.html#photutils.aperture.ApertureMask) object (for a single aperture position) or a list of `ApertureMask` objects, one for each aperture position.

The `ApertureMask` object contains a cutout of the aperture-mask weights and a [BoundingBox](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.BoundingBox.html#photutils.aperture.BoundingBox) object that provides the bounding box where the mask is to be applied.  The `ApertureMask` object is useful for extracting the data values within an aperture, either as a 1D or 2D array.

Let’s start by creating a circular aperture object.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.0
apertures = CircularAperture(positions, r=radius)

Now, let's create a list of `ApertureMask` objects using the `to_mask` method.

In [ ]:
masks = apertures.to_mask(method='exact')

Let's plot the first one, which shows a cutout of the aperture mask weights within its bounding box.  The values in the mask range from 0 (no overlap) to 1 (complete overlap).

In [ ]:
mask = masks[0]  # the first mask
plt.imshow(mask)
plt.colorbar()

We can also create a cutout from a data image over the mask bounding box. The values here are directly from the data array without any aperture weighting applied.

In [ ]:
data_cutout = mask.cutout(data)
plt.imshow(data_cutout)
plt.colorbar()

We can also create an aperture mask-weighted cutout from the data, properly handling the cases of partial or no overlap of the aperture mask with the data. Let’s plot the aperture mask weights multiplied with the data. Here the circular aperture mask has been applied to the data.

In [ ]:
data_weighted = mask.multiply(data)
plt.imshow(data_weighted)

If one only wants to get the aperture-mask-weighted data values as a 1D array, the `get_values` method can be used.

In [ ]:
mask.get_values(data)

We can also create an image with the aperture mask at its position and also plot its bounding box.

In [ ]:
maskimg = mask.to_image(shape=((200, 200)))
plt.imshow(maskimg)
mask.bbox.plot(color='red')
plt.colorbar()

In [ ]:
plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
mask.bbox.plot(color='red')
plt.title('XDF F160W Cutout');

Finally, let's use a `CircularAnnulus` aperture to create an aperture mask-weighted cutout from the data. Here the aperture mask weights have been applied to the data.

In [ ]:
from photutils.aperture import CircularAnnulus

positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
annulus_aperture = CircularAnnulus(positions, r_in=10, r_out=15)
masks = annulus_aperture.to_mask(method='exact')
plt.imshow(masks[0].multiply(data))

In [ ]:
# 1D array of aperture-weighted data values within the annulus
masks[0].get_values(data)

In [ ]:
plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.)
plt.imshow(data, norm=norm)
masks[0].bbox.plot(color='red')
plt.imshow(masks[0].to_image(data.shape), alpha=0.3)
plt.title('XDF F160W Cutout');

<div class="alert alert-warning alert-block">
<h3 style='margin-top: 0;'>Learn More</h3>

The [local background subtraction notebook](03-aperture_local_bkgsub.ipynb) covers:

- Aperture photometry with local background subtraction using CircularAnnulus apertures
  - Using the mean within a circular annulus
  - Using the sigma-clipped median within a circular annulus
</div>